<a href="https://colab.research.google.com/github/mgoncerz/SeparationPowerTool/blob/master/SP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install uproot

File loading:

In [0]:
import os
import uproot
import numpy
import awkward
import dask.dataframe

In [0]:
class FileReader:
  filename = None
  number_of_entries = None
  variable_names = []

  _read_function = None
  _file = None

  def __init__(self, filepath, root_treename = None, csv_delimiter = ','):

    self.filename, extension = os.path.splitext(filepath)

    if extension.lower() == '.root':

      self._file = uproot.open(filepath)[root_treename]
      self.variable_names = [str(branch).strip(" b'") for branch in self._file.keys()]
      self.entries = self._file.numentries
      self._read_function = self._rootGetColumn

    elif extension.lower() == '.csv' or extension.lower() == '.txt':

      self._file = dask.dataframe.read_csv(filepath, sep=csv_delimiter)
      self.variable_names = list(self._file.columns)
      self.entries = len(self._file.index)
      self._read_function = self._csvGetColumn

  def getColumn(self, name):
    return self._read_function(name)

  def _rootGetColumn(self, name):
    return self._file[name].array().astype(float)

  def _csvGetColumn(self, name):

    temp = self._file[name].compute()

    if temp.dtype == 'object':
      return awkward.fromiter([[float(val) for val in el.split(' ')] for el in temp])
    else:
      return numpy.array(temp).astype(float, copy=False)



Load both files here:

In [0]:
file1 = FileReader("zbb.root", "Zbb")
file2 = FileReader("tt.root", "Zbb")

Get names of variables present in both files:

In [0]:
all_variable_names = list(set(file1.variable_names).intersection(file2.variable_names))
print(all_variable_names)

Test whether variable has single or multiple values per event:

In [0]:
def isScalarVariable(data1, data2):
  return type(data1).__module__ == 'numpy' and data1.ndim == 1 and type(data2).__module__ == 'numpy' and data2.ndim == 1

Load data:


In [0]:
variable_names = []
variable_data1 = []
variable_data2 = []

for variable in all_variable_names:
  try:
    data1 = file1.getColumn(variable)
    data2 = file2.getColumn(variable)
  except:
    continue

  
  if isScalarVariable(data1, data2):

    #remove NaNs
    data1 = data1[~numpy.isnan(data1)]
    data2 = data2[~numpy.isnan(data2)]

    if data1.size == 0 or data2.size == 0:
      continue    

    variable_names.append(variable)
    variable_data1.append(data1)
    variable_data2.append(data2)

  else: #flatten data
    
    data1_flat = numpy.concatenate(data1).ravel()
    data2_flat = numpy.concatenate(data2).ravel()

    #remove NaNs
    data1_flat = data1_flat[~numpy.isnan(data1_flat)]
    data2_flat = data2_flat[~numpy.isnan(data2_flat)]
    
    if data1_flat.size == 0 or data2_flat.size == 0:
      continue  

    variable_names.append(variable)
    variable_data1.append(data1_flat)
    variable_data2.append(data2_flat)



Plot input data:

In [0]:
import matplotlib.pyplot as plt
import seaborn
import math

nrows = math.floor(math.sqrt(len(variable_data1)))
ncols = math.ceil(len(variable_data1) / nrows)

f, ax = plt.subplots(figsize=(50,50))
for index, [variable, data1, data2] in enumerate(zip(variable_names, variable_data1, variable_data2)):
  f.add_subplot(nrows, ncols, index + 1)
  seaborn.distplot(data1, kde=False, norm_hist = True).set_title(variable)
  seaborn.distplot(data2, kde=False, norm_hist = True).set_title(variable)


Normalize and calculate separation:

In [0]:
variable_data1_normalized = []
variable_data2_normalized = []

for variable, data1, data2 in zip(variable_names, variable_data1, variable_data2):

  minimum = min(data1.min(), data2.min())
  maximum = max(data1.max(), data2.max())
  
  diff = maximum - minimum
  if diff:
    diffinv = 1/diff
  else:
    diffinv = 1

  variable_data1_normalized.append((data1 - minimum)*diffinv)
  variable_data2_normalized.append((data2 - minimum)*diffinv)
  

Plot normalized data:

In [0]:
f, ax = plt.subplots(figsize=(50,50))
for index, [variable, data1, data2] in enumerate(zip(variable_names, variable_data1_normalized, variable_data2_normalized)):
  f.add_subplot(nrows, ncols, index + 1)
  seaborn.distplot(data1, kde=False, norm_hist = True).set_title(variable)
  seaborn.distplot(data2, kde=False, norm_hist = True).set_title(variable)

Calculate separation:

In [0]:
separation_values = []

for variable, data1, data2 in zip(variable_names, variable_data1_normalized, variable_data2_normalized):

  bincontent1, _ = numpy.histogram(data1, 100, range=[0,1])
  bincontent2, _ = numpy.histogram(data2, 100, range=[0,1])

  bincontent1_normalized = bincontent1 / sum(bincontent1)
  bincontent2_normalized = bincontent2 / sum(bincontent2)

  separation_values.append(round(100*0.5*sum(map(abs, bincontent1_normalized - bincontent2_normalized)), 2))


Sort everything by separation value:

In [0]:

separation_values_sorted, variable_names_sorted, variable_data1_sorted, variable_data2_sorted, variable_data1_normalized_sorted, variable_data2_normalized_sorted = \
zip(*sorted(zip(separation_values, variable_names, variable_data1, variable_data2, variable_data1_normalized, variable_data2_normalized), key=lambda x: x[0], reverse=True))

Print variables sorted by separation:

In [0]:
for separation, name in zip(separation_values_sorted, variable_names_sorted):
  print([separation, name])

Perform basic optimization:

1. Remove variables with separation below threshold:

In [0]:
SEPARATION_THRESHOLD = 50

In [0]:
separation_values_below_threshold, variable_names_below_threshold, variable_data1_below_threshold, variable_data2_below_threshold, variable_data1_normalized_below_threshold, variable_data2_normalized_below_threshold = \
zip(*filter(lambda x: x[0] > SEPARATION_THRESHOLD, zip(separation_values_sorted, variable_names_sorted, variable_data1_sorted, variable_data2_sorted, variable_data1_normalized_sorted, variable_data2_normalized_sorted)))

In [0]:
for separation, name in zip(separation_values_below_threshold, variable_names_below_threshold):
  print([separation, name])

2. Calculate linear correlation coefficients for both samples:

In [0]:
correlation_matrix1 = numpy.corrcoef(variable_data1_normalized_below_threshold)
correlation_matrix2 = numpy.corrcoef(variable_data2_normalized_below_threshold)


In [0]:
mask = numpy.triu(numpy.ones_like(correlation_matrix1, dtype=numpy.bool))
cmap = seaborn.diverging_palette(10, 220, as_cmap=True)


In [0]:
f, ax = plt.subplots(figsize=(20, 20))
seaborn.heatmap(correlation_matrix1, mask=mask, cmap=cmap, vmax=1, vmin=-1, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5}, xticklabels = variable_names_below_threshold, yticklabels = variable_names_below_threshold)

In [0]:
f, ax = plt.subplots(figsize=(20, 20))
seaborn.heatmap(correlation_matrix2, mask=mask, cmap=cmap, vmax=1, vmin=-1, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5}, xticklabels = variable_names_below_threshold, yticklabels = variable_names_below_threshold)

3. Reduce the number of variables based on their linear correlation:

In [0]:
DESIRED_NUMBER_OF_VARIABLES = 10
MAX_PEARSON_COEFFICIENT = 0.5

In [0]:
def getCorrelatedIndicesInRange(separation_indexed, correlation_matrix1, correlation_matrix2, DESIRED_NUMBER_OF_VARIABLES, MAX_PEARSON_COEFFICIENT):

  no_correlations_found = True
  correlated_indices = []

  #metrics are sorted descending, we go through them backwards starting at the number of desired variables (from lowest to highest separation)
  for list_index1, [global_index1, _] in reversed(list(enumerate(separation_indexed[:DESIRED_NUMBER_OF_VARIABLES]))):

    #for each variable we loop through the variables with higher separation
    for global_index2, _ in separation_indexed[:list_index1]:
      
      if math.fabs(correlation_matrix1[global_index1][global_index2]) > MAX_PEARSON_COEFFICIENT or math.fabs(correlation_matrix2[global_index1][global_index2]) > MAX_PEARSON_COEFFICIENT:

        correlated_indices.append(global_index1)
        no_correlations_found = False
        del separation_indexed[list_index1] #remove the correlated variable with lower separation, safe because we're moving in a reversed order
        break
        
  return no_correlations_found, correlated_indices

In [0]:
indices_rejected_by_correlation = []
indices_rejected_to_limit_variables = []

separation_indexed = list(enumerate(separation_values_below_threshold))

if DESIRED_NUMBER_OF_VARIABLES != -1 and len(separation_indexed) > DESIRED_NUMBER_OF_VARIABLES:

  while len(separation_indexed) > DESIRED_NUMBER_OF_VARIABLES:

    no_correlations_found, correlated_indices = getCorrelatedIndicesInRange(separation_indexed, correlation_matrix1, correlation_matrix2, DESIRED_NUMBER_OF_VARIABLES, MAX_PEARSON_COEFFICIENT)
    indices_rejected_by_correlation += correlated_indices

    if no_correlations_found:
      indices_rejected_to_limit_variables = [global_index for global_index, _ in separation_indexed[DESIRED_NUMBER_OF_VARIABLES:]]
      break

  if not no_correlations_found:
    _, correlated_indices = getCorrelatedIndicesInRange(separation_indexed, correlation_matrix1, correlation_matrix2, len(separation_indexed), MAX_PEARSON_COEFFICIENT)
    indices_rejected_by_correlation += correlated_indices
else:

  _, indices_rejected_by_correlation = getCorrelatedIndicesInRange(separation_indexed, correlation_matrix1, correlation_matrix2, len(separation_indexed), MAX_PEARSON_COEFFICIENT)

In [0]:
print('Rejected for correlation:')
for index in indices_rejected_by_correlation:
  print(variable_names_below_threshold[index])

print('')
print('')

print('Rejected to limit variables:')
for index in indices_rejected_to_limit_variables:
  print(variable_names_below_threshold[index])  

In [0]:
_, separation_values_optimized, variable_names_optimized, variable_data1_optimized, variable_data2_optimized, variable_data1_normalized_optimized, variable_data2_normalized_optimized = \
zip(*filter(lambda x: x[0] not in indices_rejected_by_correlation and x[0] not in indices_rejected_to_limit_variables, zip(range(len(separation_values_below_threshold)), separation_values_below_threshold, variable_names_below_threshold, variable_data1_below_threshold, variable_data2_below_threshold, variable_data1_normalized_below_threshold, variable_data2_normalized_below_threshold)))

In [0]:
for separation, name in zip(separation_values_optimized, variable_names_optimized):
  print([separation, name])

In [0]:
correlation_matrix1_optimized = numpy.corrcoef(variable_data1_normalized_optimized)
correlation_matrix2_optimized = numpy.corrcoef(variable_data2_normalized_optimized)

In [0]:
mask_optimized = numpy.triu(numpy.ones_like(correlation_matrix1_optimized, dtype=numpy.bool))

In [0]:
f, ax = plt.subplots(figsize=(20, 20))
seaborn.heatmap(correlation_matrix1_optimized, mask=mask_optimized, cmap=cmap, annot=correlation_matrix1_optimized, vmax=1, vmin=-1, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5}, xticklabels = variable_names_optimized, yticklabels = variable_names_optimized)

In [0]:
f, ax = plt.subplots(figsize=(20, 20))
seaborn.heatmap(correlation_matrix2_optimized, mask=mask_optimized, cmap=cmap, annot=correlation_matrix2_optimized, vmax=1, vmin=-1, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5}, xticklabels = variable_names_optimized, yticklabels = variable_names_optimized)